In [1]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from itertools import combinations 
from collections import defaultdict
import itertools
from sklearn import datasets

## Data to test your function

In [2]:
iris = datasets.load_iris()
data = iris.data[0:5]

#validating using below data:
#data = [[1,2],[9,-4],[5,6],[21,8]]

## Implementing my_pdist function and checking result with pdist

In [3]:
# run this code after importing pdist, look at its output
vector = pdist(data, metric='euclidean')

# now, make your own function
def my_pdist(data):
  return np.array([np.linalg.norm(np.array(v[0]) - np.array(v[1])) for v in list(combinations(data, 2))])

my_vector = my_pdist(data)
(my_vector == vector).all()

True

## Implementing my_squareform function and checking result with squareform

In [4]:
# run this code after importing squareform, look at its output
matrix = squareform(vector)

# now, make your own function
def my_squareform(vector):
    # length. Combination ans is len of vector, given r = 2, what is n in nCr formula? 
    l = int(np.ceil(np.sqrt(vector.shape[0] * 2))) 
    
    my_matrix = np.zeros((l,l))
    c = list(combinations(range(l), 2))
    for index, value in enumerate(c):
        my_matrix[value] = vector[index]
        my_matrix[value[::-1]] = vector[index]
    return my_matrix

my_matrix = my_squareform(my_vector)
(my_matrix == matrix).all()

True

## Implementing my_linkage function and checking result with linkage

In [10]:
# run each line below after importing linkage, look at its output
# linkage(vector, method='complete', metric='euclidean') #passing vector, instead of matrix
# linkage(vector, method='single', metric='euclidean') #passing vector, instead of matrix
# linkage(vector, method='average', metric='euclidean') #passing vector, instead of matrix

# helper function to calculate cluster distance
def clust_dist(key, baseline_distance, cluster, method):
    clust_comb = list(itertools.product(*[cluster[c] for c in key]))
    if method == 'single':
        min_dist = min([baseline_distance[tuple(sorted(x))] for x in clust_comb])
    if method == 'complete':
        min_dist = max([baseline_distance[tuple(sorted(x))] for x in clust_comb])
    if method == 'average':
        min_dist = np.mean([baseline_distance[tuple(sorted(x))] for x in clust_comb])
    return min_dist

# now, make your own function
def my_linkage(matrix, method):
    # initialize variables
    m = matrix.shape[0]
    Z = np.zeros((m-1,4))
    next_clust = m
    r,c = np.triu_indices(m,1)

    # create baseline distance dict using the matrix upper triangle and index. 
    baseline_distance = {}
    for key in zip(r,c):
        baseline_distance[key] = matrix[key]

    # create distance dict which is updated every loop
    distance = baseline_distance

    # create cluster which is updated every loop, each data point is consider as its own cluster to start with. 
    cluster = defaultdict(list)
    for i in range(matrix.shape[0]):
        cluster[i] = [i]

    # loop
    for grp_step in range(m-1):
        
        # using distance dict, find min distance and min key. 
        min_key = min(distance, key=distance.get) 
        min_distance = distance[min_key]
        
        # create Z matrix, every row is organized as item 1 and item 2 to form cluster n + 1, distance between item 1 and item 2, number of points in the formed cluster. 
        Z[grp_step,:] = min_key[0] ,min_key[1], min_distance, len(cluster[min_key[0]])+len(cluster[min_key[1]])
        
        # update cluster dict and next clust. Group points with min distance to form a new cluster and remove points that are already grouped. 
        element = [cluster.pop(min_key[0])]
        element.append(cluster.pop(min_key[1]))
        element =[item for sublist in element for item in sublist]
        cluster[next_clust] = element
        next_clust = next_clust + 1
        
        # update distance dict by using helper function clust_dict, which calculate cluster distance based on points inside the cluster. 
        distance = {}
        for key in combinations(cluster.keys(),2):
            if key in baseline_distance.keys():
                distance[key] = baseline_distance[key]
            if key not in distance.keys():
                distance[key] = clust_dist(key, baseline_distance, cluster, method)
    return(Z)

print((my_linkage(my_matrix, method = 'single') == linkage(vector, method='single')).all())
print((my_linkage(my_matrix, method = 'complete') == linkage(vector, method='complete')).all())
print((my_linkage(my_matrix, method = 'average') == linkage(vector, method='average')).all())

True
True
True
